# MNIST CNN

### Imports

In [10]:
import torch.optim as optim 
import torch.utils.data 
import torch.backends.cudnn as cudnn
import torchvision 
from torch import nn
import torch.nn.functional as F
from torchvision import transforms as transforms 
import numpy as np 
import matplotlib.pyplot as plt

from tqdm import tqdm

### Hyper Parameters 

In [11]:
lr = 0.001
epochs = 10 
train_batch_size = 10
test_batch_size = 10
num_classes = 10

In [12]:
train_transform = transforms.Compose([transforms.RandomHorizontalFlip(), transforms.ToTensor()])
test_transform = transforms.Compose([transforms.ToTensor()])
train_set = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform = train_transform)
train_loader = torch.utils.data.DataLoader(dataset = train_set, batch_size = train_batch_size, shuffle=True)
test_set = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform = test_transform)
test_loader = torch.utils.data.DataLoader(dataset = test_set, batch_size = test_batch_size, shuffle=False)

In [13]:
if torch.cuda.is_available:
    device = torch.device('cuda:0')
    print('Using GPU')
else:
    device = torch.device('cpu')
    print('Using CPU')

Using GPU


In [14]:
class ConvNet(torch.nn.Module):
    def __init__(self, num_classes):
        super(ConvNet, self).__init__()
        self.layer1= nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2), 
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2=nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2), 
            nn.BatchNorm2d(32), 
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1=nn.Linear(7*7*32, 10)
#         self.fc2=nn.Linear(1024,512)
#         self.fc3=nn.Linear(512,10)
    def forward(self, x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=out.reshape(out.size(0), -1)
#         out=F.relu(self.fc1(out))
#         out=F.relu(self.fc2(out))
#         out=F.softmax(self.fc3(out))
        out=self.fc1(out)
        return out
model = ConvNet(10).to(device)
        


In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),  lr = lr)

In [ ]:
total_step = len(train_loader)
for epoch in range(epochs):
    for i,(images, labels) in enumerate(train_loader): 
        images = images.to(device)
        labels = labels.to(device)
        
        #forward 
        outputs = model(images) 
        loss = criterion(outputs, labels) 
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if(i+1)%100==0: 
            print('Epoch [{}/{}], step [{},{}], Loss: {:.4f}'.format(epoch+1, epochs, i+1, total_step, loss.item()))
        break
    break

In [35]:
model.eval()
with torch.no_grad():
    correct = 0 
    total = 0 
    for images, labels in test_loader:
        images = images.to(device) 
        labels = labels.to(device) 
        outputs = model(images) 
        _, predicted = torch.max(outputs.data, 1)
        total+=labels.size(0)
        correct+=(predicted==labels).sum().item()
    print('test accuracy of model on 100000 images: {} %'.format(100*correct/total))
    
torch.save(model.state_dict(), 'model.ckpt')

test accuracy of model on 100000 images: 97.84 %
